<a href="https://colab.research.google.com/github/KBanerjee90/Deep-learning-with-Python/blob/master/WebsiteBrochure_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Project 1 - Create a Brochure from Website**

# Environment Creation

In [2]:
 !pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 19.2 MB/s eta 0:00:00


In [3]:
!virtualenv llms

created virtual environment CPython3.11.12.final.0-64 in 750ms
  creator CPython3Posix(dest=/content/llms, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.1.1, setuptools==80.3.1
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [4]:
 !source llms/bin/activate

In [6]:
!pip install -r /content/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 17.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 100.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 83.9 MB/s eta 0:00:00
   ━━━━━━

In [1]:
!pip install eiapy

In [2]:
import os
os.environ['CONDA_PREFIX'] = '/usr/local/envs/llms'


# New Section

In [3]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
#from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [4]:
# Load environment variables in a file called .env

#load_dotenv(override=True)
#api_key = os.getenv('OPENAI_API_KEY')
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


In [5]:
import os
os.environ['OPENAI_API_KEY'] = api_key

In [6]:
# Initialize and constants
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [7]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [8]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [9]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
#and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
#Include details of company culture, customers and careers/jobs if you have the information."

In [10]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [11]:
# get_links - function
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [12]:
# get_all_details - function
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
# get_brochure_user_prompt - function
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [15]:
# create_brochure - function defination
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [16]:
# Final call to create brochure
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/about'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


```json
{
    "links": [
        {"type": "about page", "url": "https://huggingface.co/about"},
        {"type": "careers page", "url": "https://huggingface.co/careers"},
        {"type": "enterprise page", "url": "https://huggingface.co/enterprise"}
    ]
}
```

### Hugging Face: The AI Community Building the Future

**Welcome to Hugging Face!**  
Hugging Face is your platform for artificial intelligence and machine learning collaboration. Our mission is to facilitate the machine learning community’s efforts to innovate, create, and improve.

#### Key Features:
- **Explore AI Apps**: Discover a wide array of applications powered by machine learning.
- **Browse 1M+ Models**: Access an extensive collection of models across various domains.
- **Datasets and Spaces**: Utilize numerous datasets and collaborative spaces to enhance your projects.
  
#### Collaborate with Us
Join over 50,000 organizations leveraging Hugging Face to build AI solutions that matter.

#### Open Source Tools:
We are committed to supporting the community with tools like:
- **Transformers**: Cutting-edge machine learning for PyTorch, TensorFlow, and JAX.
- **Diffusers**: State-of-the-art diffusion models.
- **Safetensors**: A secure method to distribute neural network weights.

#### Get Involved
**Careers**: Looking for an opportunity to join our dynamic team? Check our [Current Openings](https://huggingface.co/careers) to find your place in shaping the future of AI.

**Enterprise Solutions**: For organizations seeking advanced AI capabilities, explore our [Enterprise Hub](https://huggingface.co/enterprise) for tailored solutions.

**Learn More**: To understand more about our vision and community impact, visit our [About Page](https://huggingface.co/about). 

Join the revolution in machine learning and contribute to building the tools of tomorrow!

In [17]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )

    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [18]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


json
{
    "links": [
        {
            "type": "about page",
            "url": "https://huggingface.co/about"
        },
        {
            "type": "careers page",
            "url": "https://huggingface.co/jobs"
        },
        {
            "type": "documentation page",
            "url": "https://huggingface.co/docs"
        },
        {
            "type": "blog page",
            "url": "https://huggingface.co/blog"
        },
        {
            "type": "community page",
            "url": "https://huggingface.co/community"
        }
    ]
}
 

## Hugging Face Brochure

### Introduction
Hugging Face is the AI community building the future. Our platform empowers the machine learning community to collaborate on models, datasets, and applications. Join us in exploring the endless possibilities of AI!

### Key Offerings
- **Models**: Browse over 1 million models trending in various fields.
- **Datasets**: Access 250,000+ datasets to enhance your machine learning projects.
- **Spaces**: Explore 400,000+ applications built on our platform.

### Why Hugging Face?
- **Collaboration**: Host and collaborate on unlimited public models, datasets, and applications.
- **Flexibility**: Work with a variety of modalities - text, image, video, audio, or even 3D.
- **Professional Development**: Build and showcase your portfolio while sharing your work with the community.

### Get Started
- **Enterprise Solutions**: Experience our enterprise-grade platform with enhanced security and support.
- **Sign Up**: Join the thriving community today!

### Connect with Us
- **About Us**: Learn more about our mission and values [here](https://huggingface.co/about).
- **Careers**: Explore job opportunities with us [here](https://huggingface.co/jobs).
- **Documentation**: Read our comprehensive documentation [here](https://huggingface.co/docs).
- **Blog**: Stay updated with the latest news and articles on our [blog](https://huggingface.co/blog).
- **Community**: Join our community discussions [here](https://huggingface.co/community). 

Join us at Hugging Face and be a part of the AI revolution!